In [ ]:
# h5 file opener

import h5py
import numpy as np
import cv2
import matplotlib.pyplot as plt

hdf5_path = r"/Users/cadenmyers/billingelab/dev/skyrmion-lattices/December_extracted_data.h5"

def open_hdf5(file_path):
    """ Open an HDF5 file and return the file object. """
    return h5py.File(file_path, 'r')

hdf5_file = open_hdf5(hdf5_path)

#print(hdf5_file)

In [ ]:
# Gaussian blur

def preprocess_image(image):
    # Apply Gaussian smoothing
    return cv2.GaussianBlur(image, (5, 5), 0)

#print(preprocess_image(intensity_vals))


In [ ]:
# Extraction of data for numors

def get_data_for_ml(hdf5_file, numor_ids):
    """ Extract data and metadata for ML/AI applications for one or multiple numors. """
    all_data = {}
    
    # Ensure numor_ids is a list for single and multiple numor handling
    if not isinstance(numor_ids, list):
        numor_ids = [numor_ids]
    
    for numor_id in numor_ids:
        group = hdf5_file[f'Numor{numor_id}']
        data = {
            'q_x': group['q_x (inverse angstrom)'][:],
            'q_y': group['q_y (inverse angstrom)'][:],
            'intensity': group['intensity (counts per standard monitor)'][:],
            'intensity_err': group['intensity_err (counts per standard monitor)'][:]
        }
        
        # Retrieve and include metadata in the dictionary
        metadata = {attr: group.attrs[attr] for attr in group.attrs}
        data.update(metadata)
        
        # Store data for this numor keyed by its ID
        all_data[numor_id] = data
    
    return all_data

#print(get_data_for_ml(hdf5_file, start_numor))

In [ ]:
# Defines hexagon for a specified center, radius, and rotation angle

import numpy as np

def hexagon_vertices(center, radius, angle):
    vertices = []
    for i in range(6):
        theta = np.radians(60 * i + angle)
        x = center[0] + radius * np.cos(theta)
        y = center[1] + radius * np.sin(theta)
        vertices.append((x, y))
    return np.array(vertices)

#center = (0, 0)
#radius = 1
#angle = 0
#print(hexagon_vertices(center, radius, angle))
#vertices = hexagon_vertices(center, radius, angle)
#plt.figure()
#plt.plot(*zip(*vertices), marker='o')
#plt.xlim(-1, 1)
#plt.ylim(-1, 1)
#plt.gca().set_aspect('equal', adjustable='box')
#plt.show()

In [ ]:
# Find hexagon orientation that optimizes intensity at vertices

def evaluate_orientation(image, center, radius, vertex_area=8):
    max_intensity_sum = 0
    best_angle = 0
    half_area = vertex_area // 2

    for angle in range(0, 360, 1):  # 1 degree steps
        vertices = hexagon_vertices(center, radius, angle)
        intensity_sum = 0
        
        for vertex in vertices:
            x, y = int(vertex[0]), int(vertex[1])
            if 0 <= x < image.shape[1] and 0 <= y < image.shape[0]:
                x_min = max(0, x - half_area)
                x_max = min(image.shape[1], x + half_area + 1)
                y_min = max(0, y - half_area)
                y_max = min(image.shape[0], y + half_area + 1)
                intensity_sum += np.sum(image[y_min:y_max, x_min:x_max])

        if intensity_sum > max_intensity_sum:
            max_intensity_sum = intensity_sum
            best_angle = angle

    return best_angle, max_intensity_sum